In [11]:
# Importing

# modules
import csv
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import datetime

# functions
from implementations import *
from helpers import *

#data
(labels, data, ids) = load_csv_data("data/train.csv")  # load data

In [12]:
# Function that returns tuple (data_processed and new_labels)

def process_data(data, labels, sample_filtering=True, feature_filtering=True):
    # Preparing data

    # set values of -999 to NaN. 
    # Then calculate the means of the features. 
    # Replace NaN values with new values.

    data_process = np.array(data[:,:])
    labels_select = np.array(labels[:])
    lab = []
    for entry in labels_select:
        if int(entry) == 1:
            lab.append(1)
        else:
            lab.append(0)
    lab = np.array(lab)
    data_process[data_process == -999] = np.nan

    # Filtering weak features and samples

    # retrieve percentage for each feature 
    # - how many nan's are there?
    
    if feature_filtering:
        nan_count = []
        for c in data_process.T:
            count = 0
            for e in c:
                if np.isnan(e):
                    count += 1
            pcent = count / data_process.shape[0]
            nan_count.append(pcent)

        # filter out features which have nan values of 50%

        data_set_filtered = []
        for idx, entry in enumerate(nan_count):
            if entry < 0.6:
                #append the column of the original dataset that is good
                data_set_filtered.append(data_process.T[idx]) 
        #save that shit as an np array
        data_set_filtered = np.array(data_set_filtered).T #save that shit as an np array
        
    if sample_filtering:
        nan_count_2 = []
        data_set_filtered_2 = [] # dataset filtered for columns and samples
        y = [] # array that gets rid of entries that are no longer corresponding in the dataframe
        for sample in data_set_filtered:
            count = 0
            for col in sample:
                if np.isnan(col):
                    count += 1
            pcent = count / data_set_filtered.shape[1]
            nan_count_2.append(pcent)

        for idx, entry in enumerate(nan_count_2):
            if entry < 0.15:
                y.append(lab[idx])
                data_set_filtered_2.append(data_set_filtered[idx])
        data_set_filtered_2 = np.array(data_set_filtered_2) # turn dat shit into an array
        y = np.array(y) # also this one gotta be an array


    # print new dimensions of the dataframe after filtering

    print('The original dimensions of the training data set was {0} samples'
          ' and {1} columns. After feature and sample filtering, there are'
          ' {2} samples and {3} columns'.format(data_process.shape[0],
                                                data_process.shape[1],
                                                data_set_filtered_2.shape[0],
                                                data_set_filtered_2.shape[1]))

    # Getting Rid of NAN and Replacing with Mean

    # variable reassigned
    data_nan = data_set_filtered_2.copy() 
    # create list with average values of columns, excluding nans
    column_means = [] 
    for column in data_nan.T:
        column_means.append(np.nanmean(column))
    # variable containing locations of nan in data frame
    inds = np.where(np.isnan(data_nan)) 
    # reassign locations of nan to the column means
    data_nan[inds] = np.take(column_means, inds[1])
    return (data_nan, y)

def standardize(data):
     # standardize and normalize the features
    for idx, column in enumerate(data.T):
        mean = np.mean(column)
        variance = np.std(column)
        for entry, value in enumerate(column):
            data[entry, idx] = (value - mean)/variance
    return data

(new_data, new_labels) = process_data(data, labels)
data_std = standardize(new_data)

# Shuffle data and create training and testing sections
test_ratio_ = 0.3
X_train, y_train, X_test, y_test = test_train(data_std, new_labels, test_ratio = test_ratio_)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

print('Standardized and randomized samples are found as the variables X_train, y_train, X_test, y_test. Values' +
      ' are split for testing and training sets with the test_ratio of {}'.format(test_ratio_))

The original dimensions of the training data set was 250000 samples and 30 columns. After feature and sample filtering, there are 223877 samples and 23 columns
Standardized and randomized samples are found as the variables X_train, y_train, X_test, y_test. Values are split for testing and training sets with the test_ratio of 0.3
